In [160]:
# Packages / libraries
import os #provides functions for interacting with the operating system
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime

%matplotlib inline

# To install sklearn type "pip install numpy scipy scikit-learn" to the anaconda terminal

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

# Increases the size of sns plots
sns.set(rc={'figure.figsize':(12,10)})

# import sys
# !conda list Check the packages installed

In [161]:
# Loading the cumulative raw data

raw_data_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
raw_data_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
raw_data_Recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
raw_data_vaccine = pd.read_csv('https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_doses_admin_global.csv')

print("The Shape of Cornirmed is: ", raw_data_confirmed.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered.shape)
print("The Shape of Cornirmed is: ", raw_data_vaccine.shape)

raw_data_vaccine.head()

The Shape of Cornirmed is:  (279, 607)
The Shape of Cornirmed is:  (279, 607)
The Shape of Cornirmed is:  (264, 607)
The Shape of Cornirmed is:  (633, 290)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2021-09-06,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15
0,4.0,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.9391,67.7100,...,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0
1,8.0,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.1533,20.1683,...,1499898.0,1539232.0,1539232.0,1567313.0,1567313.0,1579402.0,1598196.0,1601552.0,1610157.0,1618111.0
2,12.0,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.0339,1.6596,...,8000000.0,8000000.0,8000000.0,8000000.0,8000000.0,8000000.0,8000000.0,9989662.0,9989662.0,9989662.0
3,20.0,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.5063,1.5218,...,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0
4,24.0,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.2027,17.8739,...,2086325.0,2086325.0,2086325.0,2086325.0,2086325.0,2086325.0,2086325.0,2469333.0,2469333.0,2469333.0


In [162]:
# Drop unecessary columns
raw_data_vaccine = raw_data_vaccine.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population'], axis=1)
raw_data_vaccine.rename(columns={'Province_State': 'Province/State', 'Country_Region': 'Country/Region', 'Long_': 'Long'}, inplace=True)
raw_data_vaccine.head()

,Province/State,Country/Region,Lat,Long,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,...,2021-09-06,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15
0,NaN,Afghanistan,33.9391,67.7100,NaN,NaN,NaN,NaN,NaN,NaN,...,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0,1979652.0
1,NaN,Albania,41.1533,20.1683,NaN,NaN,NaN,NaN,NaN,NaN,...,1499898.0,1539232.0,1539232.0,1567313.0,1567313.0,1579402.0,1598196.0,1601552.0,1610157.0,1618111.0
2,NaN,Algeria,28.0339,1.6596,0.0,0.0,0.0,0.0,0.0,0.0,...,8000000.0,8000000.0,8000000.0,8000000.0,8000000.0,8000000.0,8000000.0,9989662.0,9989662.0,9989662.0
3,NaN,Andorra,42.5063,1.5218,0.0,0.0,0.0,0.0,0.0,0.0,...,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0,93430.0
4,NaN,Angola,-11.2027,17.8739,NaN,NaN,NaN,NaN,NaN,NaN,...,2086325.0,2086325.0,2086325.0,2086325.0,2086325.0,2086325.0,2086325.0,2469333.0,2469333.0,2469333.0


In [163]:
# Un-Pivoting the data

raw_data_confirmed2 = pd.melt(raw_data_confirmed, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_deaths2 = pd.melt(raw_data_deaths, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_Recovered2 = pd.melt(raw_data_Recovered, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])
raw_data_vaccine2 = pd.melt(raw_data_vaccine, id_vars =['Province/State', 'Country/Region', 'Lat', 'Long'], var_name=['Date'])

print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered2.shape)
print("The Shape of Cornirmed is: ", raw_data_vaccine2.shape)

raw_data_vaccine2.head()

The Shape of Cornirmed is:  (168237, 6)
The Shape of Cornirmed is:  (168237, 6)
The Shape of Cornirmed is:  (159192, 6)
The Shape of Cornirmed is:  (175974, 6)


,Province/State,Country/Region,Lat,Long,Date,value
0,NaN,Afghanistan,33.9391,67.7100,2020-12-12,NaN
1,NaN,Albania,41.1533,20.1683,2020-12-12,NaN
2,NaN,Algeria,28.0339,1.6596,2020-12-12,0.0
3,NaN,Andorra,42.5063,1.5218,2020-12-12,0.0
4,NaN,Angola,-11.2027,17.8739,2020-12-12,NaN


In [164]:
raw_data_confirmed2.head()

,Province/State,Country/Region,Lat,Long,Date,value
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [165]:
raw_data_deaths2.head()

,Province/State,Country/Region,Lat,Long,Date,value
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [166]:
raw_data_Recovered2.head()

,Province/State,Country/Region,Lat,Long,Date,value
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [167]:
# Converting the new column to dates

raw_data_confirmed2['Date'] = pd.to_datetime(raw_data_confirmed2['Date'])
raw_data_deaths2['Date'] = pd.to_datetime(raw_data_deaths2['Date'])
raw_data_Recovered2['Date'] = pd.to_datetime(raw_data_Recovered2['Date'])
raw_data_vaccine2['Date'] = pd.to_datetime(raw_data_vaccine2['Date'])

In [168]:
# Renaming the Values
raw_data_confirmed2.columns = raw_data_confirmed2.columns.str.replace('value', 'Confirmed')
raw_data_deaths2.columns = raw_data_deaths2.columns.str.replace('value', 'Deaths')
raw_data_Recovered2.columns = raw_data_Recovered2.columns.str.replace('value', 'Recovered')
raw_data_vaccine2.columns = raw_data_vaccine2.columns.str.replace('value', 'Vaccines')

In [169]:
# Investigating the NULL values
raw_data_Recovered2.isnull().sum()

Province/State    116379
Country/Region         0
Lat                  603
Long                 603
Date                   0
Recovered              0
dtype: int64

In [170]:
# Investigating the NULL values
raw_data_deaths2.isnull().sum()

Province/State    115776
Country/Region         0
Lat                 1206
Long                1206
Date                   0
Deaths                 0
dtype: int64

In [171]:
raw_data_confirmed2.isnull().sum()

Province/State    115776
Country/Region         0
Lat                 1206
Long                1206
Date                   0
Confirmed              0
dtype: int64

In [172]:
raw_data_vaccine2.isnull().sum()

Province/State     45314
Country/Region         0
Lat               120374
Long              120374
Date                   0
Vaccines           98160
dtype: int64

In [173]:
# Dealing with NULL values

raw_data_confirmed2['Province/State'].fillna(raw_data_confirmed2['Country/Region'], inplace=True)
raw_data_deaths2['Province/State'].fillna(raw_data_deaths2['Country/Region'], inplace=True)
raw_data_Recovered2['Province/State'].fillna(raw_data_Recovered2['Country/Region'], inplace=True)
raw_data_vaccine2['Province/State'].fillna(raw_data_vaccine2['Country/Region'], inplace=True)

In [174]:
# Deleting the NULL values
raw_data_Recovered2 = raw_data_Recovered2.dropna(subset = ['Lat'])
raw_data_deaths2 = raw_data_deaths2.dropna(subset = ['Lat'])
raw_data_confirmed2 = raw_data_confirmed2.dropna(subset = ['Lat'])
raw_data_vaccine2 = raw_data_vaccine2.dropna(subset = ['Lat'])

# printing shapes before the join
print("The Shape of Cornirmed is: ", raw_data_confirmed2.shape)
print("The Shape of Cornirmed is: ", raw_data_deaths2.shape)
print("The Shape of Cornirmed is: ", raw_data_Recovered2.shape)
print("The Shape of Cornirmed is: ", raw_data_vaccine2.shape)

raw_data_vaccine2.head()

The Shape of Cornirmed is:  (167031, 6)
The Shape of Cornirmed is:  (167031, 6)
The Shape of Cornirmed is:  (158589, 6)
The Shape of Cornirmed is:  (55600, 6)


,Province/State,Country/Region,Lat,Long,Date,Vaccines
0,Afghanistan,Afghanistan,33.9391,67.7100,2020-12-12,NaN
1,Albania,Albania,41.1533,20.1683,2020-12-12,NaN
2,Algeria,Algeria,28.0339,1.6596,2020-12-12,0.0
3,Andorra,Andorra,42.5063,1.5218,2020-12-12,0.0
4,Angola,Angola,-11.2027,17.8739,2020-12-12,NaN


In [175]:
# Full Joins

# Confirmed with Deaths
full_join = raw_data_confirmed2.merge(raw_data_deaths2[['Province/State','Country/Region','Date','Deaths']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

print("Shape of first join: ", full_join.shape)

# full join with Recovered
full_join = full_join.merge(raw_data_Recovered2[['Province/State','Country/Region','Date','Recovered']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State', 'Country/Region','Date'])

# full join with Vaccine
full_join = full_join.merge(raw_data_vaccine2[['Province/State','Country/Region','Date','Vaccines']], 
                                      how = 'left', 
                                      left_on = ['Province/State','Country/Region','Date'], 
                                      right_on = ['Province/State','Country/Region','Date'])

print("Shape of second join: ", full_join.shape)

full_join.tail()

Shape of first join:  (167031, 7)
Shape of second join:  (167031, 9)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Vaccines
167026,Vietnam,Vietnam,14.058324,108.277199,2021-09-15,645640,16186,0.0,31254856.0
167027,West Bank and Gaza,West Bank and Gaza,31.952200,35.233200,2021-09-15,377134,3856,0.0,NaN
167028,Yemen,Yemen,15.552727,48.516388,2021-09-15,8527,1612,0.0,NaN
167029,Zambia,Zambia,-13.133897,27.849332,2021-09-15,208161,3636,0.0,NaN
167030,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-09-15,127083,4551,0.0,4855816.0


In [176]:
# checking for null values (especially long and lat)
full_join.isnull().sum()

Province/State         0
Country/Region         0
Lat                    0
Long                   0
Date                   0
Confirmed              0
Deaths                 0
Recovered           9045
Vaccines          131260
dtype: int64

In [177]:
# Adding Month and Year as a new Column
full_join['Month-Year'] = full_join['Date'].dt.strftime('%b-%Y')

full_join.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Vaccines,Month-Year
167026,Vietnam,Vietnam,14.058324,108.277199,2021-09-15,645640,16186,0.0,31254856.0,Sep-2021
167027,West Bank and Gaza,West Bank and Gaza,31.952200,35.233200,2021-09-15,377134,3856,0.0,NaN,Sep-2021
167028,Yemen,Yemen,15.552727,48.516388,2021-09-15,8527,1612,0.0,NaN,Sep-2021
167029,Zambia,Zambia,-13.133897,27.849332,2021-09-15,208161,3636,0.0,NaN,Sep-2021
167030,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-09-15,127083,4551,0.0,4855816.0,Sep-2021


In [178]:
#############################################################################################
######################## Braking the numbers by Day #########################################
#############################################################################################

## Applying it on all dataset

#creating a new df    
full_join2 = full_join.copy()

#creating a new date columns - 1
full_join2['Date - 1'] = full_join2['Date'] + pd.Timedelta(days=1)
full_join2.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1', 
                           'Vaccines': 'Vaccines - 1', 'Date': 'Date Minus 1'}, inplace=True)

#Joing on the 2 DFs
full_join3 = full_join.merge(full_join2[['Province/State', 'Country/Region','Confirmed - 1', 'Deaths - 1', 
                            'Recovered - 1', 'Vaccines - 1', 'Date - 1', 'Date Minus 1']], how = 'left',
                             left_on = ['Province/State','Country/Region','Date'], 
                             right_on = ['Province/State', 'Country/Region','Date - 1'])

#minus_onedf.rename(columns={'Confirmed': 'Confirmed - 1', 'Deaths': 'Deaths - 1', 'Recovered': 'Recovered - 1'}, inplace=True)

full_join3.head()

# Additional Calculations
full_join3['Confirmed Daily'] = full_join3['Confirmed'] - full_join3['Confirmed - 1']
full_join3['Deaths Daily'] = full_join3['Deaths'] - full_join3['Deaths - 1']
full_join3['Recovered Daily'] = full_join3['Recovered'] - full_join3['Recovered - 1']
full_join3['Vaccines Daily'] = full_join3['Vaccines'] - full_join3['Vaccines - 1']

print(full_join3.shape)
full_join3.head()

(167031, 20)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Vaccines,Month-Year,Confirmed - 1,Deaths - 1,Recovered - 1,Vaccines - 1,Date - 1,Date Minus 1,Confirmed Daily,Deaths Daily,Recovered Daily,Vaccines Daily
0,Afghanistan,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0.0,NaN,Jan-2020,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
1,Albania,Albania,41.15330,20.168300,2020-01-22,0,0,0.0,NaN,Jan-2020,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
2,Algeria,Algeria,28.03390,1.659600,2020-01-22,0,0,0.0,NaN,Jan-2020,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
3,Andorra,Andorra,42.50630,1.521800,2020-01-22,0,0,0.0,NaN,Jan-2020,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN
4,Angola,Angola,-11.20270,17.873900,2020-01-22,0,0,0.0,NaN,Jan-2020,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN


In [179]:
# Additing manually the numbers for first day

full_join3['Confirmed Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Confirmed']
full_join3['Deaths Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Deaths']
full_join3['Recovered Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Recovered']
full_join3['Vaccines Daily'].loc[full_join3['Date'] == '2020-01-22'] = full_join3['Vaccines']

# deleting columns
del full_join3['Confirmed - 1']
del full_join3['Deaths - 1']
del full_join3['Recovered - 1']
del full_join3['Date - 1']
del full_join3['Date Minus 1']
del full_join3['Vaccines - 1']

C:\Users\Tuta\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [180]:
full_join3.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Vaccines,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily,Vaccines Daily
167026,Vietnam,Vietnam,14.058324,108.277199,2021-09-15,645640,16186,0.0,31254856.0,Sep-2021,10585.0,250.0,0.0,905936.0
167027,West Bank and Gaza,West Bank and Gaza,31.952200,35.233200,2021-09-15,377134,3856,0.0,NaN,Sep-2021,2366.0,19.0,0.0,NaN
167028,Yemen,Yemen,15.552727,48.516388,2021-09-15,8527,1612,0.0,NaN,Sep-2021,25.0,4.0,0.0,NaN
167029,Zambia,Zambia,-13.133897,27.849332,2021-09-15,208161,3636,0.0,NaN,Sep-2021,112.0,1.0,0.0,NaN
167030,Zimbabwe,Zimbabwe,-19.015438,29.154857,2021-09-15,127083,4551,0.0,4855816.0,Sep-2021,266.0,1.0,0.0,55055.0


In [181]:
os. getcwd()

'C:\\Users\\Tuta\\Documents\\Biblioteca do Engenheiro\\Linguagem Computacional\\Python\\Data science\\Covid-19'

In [182]:
# Replacing dot for coma from columns names for POWER BI analysis
full_join3['Lat'] = full_join3['Lat'].astype(str)
full_join3['Lat'] = full_join3['Lat'].str.replace(".",",")

full_join3['Long'] = full_join3['Long'].astype(str)
full_join3['Long'] = full_join3['Long'].str.replace(".",",")

full_join3['Recovered'] = full_join3['Recovered'].astype(str)
full_join3['Recovered'] = full_join3['Recovered'].str.replace(".",",")

full_join3['Vaccines'] = full_join3['Vaccines'].astype(str)
full_join3['Vaccines'] = full_join3['Vaccines'].str.replace(".",",")

full_join3['Confirmed Daily'] = full_join3['Confirmed Daily'].astype(str)
full_join3['Confirmed Daily'] = full_join3['Confirmed Daily'].str.replace(".",",")

full_join3['Deaths Daily'] = full_join3['Deaths Daily'].astype(str)
full_join3['Deaths Daily'] = full_join3['Deaths Daily'].str.replace(".",",")

full_join3['Recovered Daily'] = full_join3['Recovered Daily'].astype(str)
full_join3['Recovered Daily'] = full_join3['Recovered Daily'].str.replace(".",",")

full_join3['Vaccines Daily'] = full_join3['Vaccines Daily'].astype(str)
full_join3['Vaccines Daily'] = full_join3['Vaccines Daily'].str.replace(".",",")

<ipython-input-182-e99620125344>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  full_join3['Lat'] = full_join3['Lat'].str.replace(".",",")
<ipython-input-182-e99620125344>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  full_join3['Long'] = full_join3['Long'].str.replace(".",",")
<ipython-input-182-e99620125344>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  full_join3['Recovered'] = full_join3['Recovered'].str.replace(".",",")
<ipython-input-182-e99620125344>:12: FutureWarning: The default value of regex will change f

In [183]:
# Exporting the data

file_name = Covid_Cases.csv
full_join3.to_csv(file_name, sep='\t', index=False)
full_join3.tail()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Vaccines,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily,Vaccines Daily
167026,Vietnam,Vietnam,"14,058324","108,277199",2021-09-15,645640,16186,"0,0","31254856,0",Sep-2021,"10585,0","250,0","0,0","905936,0"
167027,West Bank and Gaza,West Bank and Gaza,"31,9522","35,2332",2021-09-15,377134,3856,"0,0",nan,Sep-2021,"2366,0","19,0","0,0",nan
167028,Yemen,Yemen,"15,552727","48,516388",2021-09-15,8527,1612,"0,0",nan,Sep-2021,"25,0","4,0","0,0",nan
167029,Zambia,Zambia,"-13,133897","27,849332",2021-09-15,208161,3636,"0,0",nan,Sep-2021,"112,0","1,0","0,0",nan
167030,Zimbabwe,Zimbabwe,"-19,015438","29,154857",2021-09-15,127083,4551,"0,0","4855816,0",Sep-2021,"266,0","1,0","0,0","55055,0"


In [184]:
full_join3.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Vaccines,Month-Year,Confirmed Daily,Deaths Daily,Recovered Daily,Vaccines Daily
0,Afghanistan,Afghanistan,"33,93911","67,709953",2020-01-22,0,0,"0,0",nan,Jan-2020,"0,0","0,0","0,0",nan
1,Albania,Albania,"41,1533","20,1683",2020-01-22,0,0,"0,0",nan,Jan-2020,"0,0","0,0","0,0",nan
2,Algeria,Algeria,"28,0339","1,6596",2020-01-22,0,0,"0,0",nan,Jan-2020,"0,0","0,0","0,0",nan
3,Andorra,Andorra,"42,5063","1,5218",2020-01-22,0,0,"0,0",nan,Jan-2020,"0,0","0,0","0,0",nan
4,Angola,Angola,"-11,2027","17,8739",2020-01-22,0,0,"0,0",nan,Jan-2020,"0,0","0,0","0,0",nan
